# Loop 19 LB Feedback Analysis

## Key Results
- Submitted exp_019 (021_cpp_sa_2M_iter): CV 70.6279 | LB 70.6279
- CV-LB gap: 0.0000 (perfect alignment - this is a deterministic optimization problem)
- Target: 68.919154 | Gap: 1.709 (2.42%)

## Critical Insight: N=88 Improvement
The C++ SA with 2M iterations found a SIGNIFICANT improvement for N=88:
- Side length: 5.5548 -> 5.5357 (0.34% reduction)
- Score contribution: 0.002398 improvement

This is 100x larger than other improvements! Why?

In [1]:
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
from shapely.affinity import rotate, translate
import matplotlib.pyplot as plt

TREE_TEMPLATE = [
    (0.0, 0.8), (0.125, 0.5), (0.0625, 0.5), (0.2, 0.25), (0.1, 0.25),
    (0.35, 0.0), (0.075, 0.0), (0.075, -0.2), (-0.075, -0.2), (-0.075, 0.0),
    (-0.35, 0.0), (-0.1, 0.25), (-0.2, 0.25), (-0.0625, 0.5), (-0.125, 0.5)
]

def parse_s_value(val):
    if isinstance(val, str):
        if val.startswith('s'):
            return float(val[1:])
        return float(val)
    return float(val)

def create_tree_polygon(x, y, angle):
    tree = Polygon(TREE_TEMPLATE)
    tree = rotate(tree, angle, origin=(0, 0), use_radians=False)
    tree = translate(tree, x, y)
    return tree

def get_n_side(df, n):
    group = df[df['n'] == n]
    all_x = []
    all_y = []
    for _, row in group.iterrows():
        tree = create_tree_polygon(row['x'], row['y'], row['deg'])
        minx, miny, maxx, maxy = tree.bounds
        all_x.extend([minx, maxx])
        all_y.extend([miny, maxy])
    if not all_x:
        return 0
    return max(max(all_x) - min(all_x), max(all_y) - min(all_y))

# Load current best
df = pd.read_csv('/home/submission/submission.csv')
df['x'] = df['x'].apply(parse_s_value)
df['y'] = df['y'].apply(parse_s_value)
df['deg'] = df['deg'].apply(parse_s_value)
df['n'] = df['id'].apply(lambda x: int(x.split('_')[0]))

print(f"Loaded {len(df)} rows")

Loaded 20100 rows


In [2]:
# Calculate score breakdown by N
scores = []
for n in range(1, 201):
    s = get_n_side(df, n)
    score = (s ** 2) / n
    scores.append({'n': n, 'side': s, 'score': score})

scores_df = pd.DataFrame(scores)
print(f"Total score: {scores_df['score'].sum():.6f}")
print(f"\nTop 10 N values by score contribution:")
print(scores_df.nlargest(10, 'score')[['n', 'side', 'score']])

Total score: 70.627896

Top 10 N values by score contribution:
     n      side     score
0    1  0.813173  0.661250
1    2  0.949504  0.450779
2    3  1.142031  0.434745
4    5  1.443692  0.416850
3    4  1.290806  0.416545
6    7  1.673104  0.399897
5    6  1.548438  0.399610
8    9  1.867280  0.387415
7    8  1.755921  0.385407
14  15  2.377955  0.376978


In [3]:
# Calculate theoretical minimum (efficiency analysis)
# Theoretical minimum is when all trees are packed with 100% efficiency
# Tree area = 0.35 (from polygon calculation)

tree_area = Polygon(TREE_TEMPLATE).area
print(f"Tree area: {tree_area:.6f}")

# For N trees, minimum square side = sqrt(N * tree_area)
scores_df['min_side'] = np.sqrt(scores_df['n'] * tree_area)
scores_df['efficiency'] = (scores_df['min_side'] / scores_df['side']) ** 2 * 100

print(f"\nEfficiency analysis (how close to theoretical minimum):")
print(f"Average efficiency: {scores_df['efficiency'].mean():.2f}%")
print(f"Min efficiency: {scores_df['efficiency'].min():.2f}% at N={scores_df.loc[scores_df['efficiency'].idxmin(), 'n']}")
print(f"Max efficiency: {scores_df['efficiency'].max():.2f}% at N={scores_df.loc[scores_df['efficiency'].idxmax(), 'n']}")

# Find N values with lowest efficiency (most room for improvement)
print(f"\nN values with LOWEST efficiency (most room for improvement):")
print(scores_df.nsmallest(10, 'efficiency')[['n', 'side', 'score', 'efficiency']])

Tree area: 0.245625

Efficiency analysis (how close to theoretical minimum):
Average efficiency: 69.86%
Min efficiency: 37.15% at N=1
Max efficiency: 74.44% at N=181

N values with LOWEST efficiency (most room for improvement):
     n      side     score  efficiency
0    1  0.813173  0.661250   37.145558
1    2  0.949504  0.450779   54.488985
2    3  1.142031  0.434745   56.498619
4    5  1.443692  0.416850   58.924131
3    4  1.290806  0.416545   58.967240
6    7  1.673104  0.399897   61.422135
5    6  1.548438  0.399610   61.466135
8    9  1.867280  0.387415   63.400999
7    8  1.755921  0.385407   63.731287
14  15  2.377955  0.376978   65.156316


In [4]:
# Analyze N=88 specifically
n88 = scores_df[scores_df['n'] == 88].iloc[0]
print(f"N=88 Analysis:")
print(f"  Current side: {n88['side']:.6f}")
print(f"  Current score: {n88['score']:.6f}")
print(f"  Efficiency: {n88['efficiency']:.2f}%")
print(f"  Theoretical min side: {n88['min_side']:.6f}")

# What's special about N=88?
print(f"\nN=88 factorization: 88 = 8 × 11")
print(f"This suggests a rectangular grid arrangement might work well.")

# Check other 'grid-friendly' N values
grid_friendly = [n for n in range(1, 201) if any(
    n == a * b for a in range(2, 15) for b in range(a, 15) if a * b == n
)]
print(f"\nGrid-friendly N values (products of small integers):")
print(grid_friendly[:20])

N=88 Analysis:
  Current side: 5.535741
  Current score: 0.348232
  Efficiency: 70.53%
  Theoretical min side: 4.649193

N=88 factorization: 88 = 8 × 11
This suggests a rectangular grid arrangement might work well.

Grid-friendly N values (products of small integers):
[4, 6, 8, 9, 10, 12, 14, 15, 16, 18, 20, 21, 22, 24, 25, 26, 27, 28, 30, 32]


In [5]:
# Calculate gap to target for each N
target = 68.919154
current_total = scores_df['score'].sum()
gap = current_total - target

print(f"Current total: {current_total:.6f}")
print(f"Target: {target:.6f}")
print(f"Gap: {gap:.6f} ({100*gap/current_total:.2f}%)")

# How much improvement needed per N on average?
print(f"\nAverage improvement needed per N: {gap/200:.6f}")

# Which N values contribute most to the gap?
# Assume we need to reduce each N's score proportionally
scores_df['gap_contribution'] = scores_df['score'] * (gap / current_total)
print(f"\nN values with largest gap contribution:")
print(scores_df.nlargest(10, 'gap_contribution')[['n', 'side', 'score', 'gap_contribution']])

Current total: 70.627896
Target: 68.919154
Gap: 1.708742 (2.42%)

Average improvement needed per N: 0.008544

N values with largest gap contribution:
     n      side     score  gap_contribution
0    1  0.813173  0.661250          0.015998
1    2  0.949504  0.450779          0.010906
2    3  1.142031  0.434745          0.010518
4    5  1.443692  0.416850          0.010085
3    4  1.290806  0.416545          0.010078
6    7  1.673104  0.399897          0.009675
5    6  1.548438  0.399610          0.009668
8    9  1.867280  0.387415          0.009373
7    8  1.755921  0.385407          0.009324
14  15  2.377955  0.376978          0.009120


In [6]:
# Key insight: N=88 improved by 0.34% with 2M iterations
# If we can find 10 more N values with similar improvement potential,
# that's 0.024 total improvement (1.4% of the gap)

# Let's identify N values that might have similar characteristics to N=88:
# 1. Products of small primes (grid-friendly)
# 2. Low efficiency (room for improvement)
# 3. High score contribution (high impact)

scores_df['priority'] = (100 - scores_df['efficiency']) * scores_df['score']
print("N values with highest improvement priority (low efficiency × high score):")
print(scores_df.nlargest(15, 'priority')[['n', 'side', 'score', 'efficiency', 'priority']])

N values with highest improvement priority (low efficiency × high score):
     n      side     score  efficiency   priority
0    1  0.813173  0.661250   37.145558  41.562500
1    2  0.949504  0.450779   54.488985  20.515418
2    3  1.142031  0.434745   56.498619  18.912014
4    5  1.443692  0.416850   58.924131  17.122459
3    4  1.290806  0.416545   58.967240  17.091985
6    7  1.673104  0.399897   61.422135  15.427155
5    6  1.548438  0.399610   61.466135  15.398529
8    9  1.867280  0.387415   63.400999  14.179003
7    8  1.755921  0.385407   63.731287  13.978225
14  15  2.377955  0.376978   65.156316  13.135304
9   10  1.940696  0.376630   65.216518  13.100504
20  21  2.811667  0.376451   65.247553  13.082590
19  20  2.742469  0.376057   65.315921  13.043186
21  22  2.873270  0.375258   65.454949  12.963310
10  11  2.030803  0.374924   65.513326  12.929872


## Conclusions

1. **CV-LB alignment is perfect** - this is a deterministic optimization problem
2. **N=88 showed 100x more improvement** than other N values with 2M SA iterations
3. **Low efficiency N values** have the most room for improvement
4. **Gap to target is 1.709 points (2.42%)** - need fundamentally different approaches

## Next Steps
1. Run longer SA (5M+ iterations) on high-priority N values
2. Try asymmetric solutions (not grid-based)
3. Investigate why N=88 improved so much - can we replicate for other N?